In [1]:
import os, joblib, pandas as pd
from scipy.sparse import load_npz

BASE_DIR = r"C:\Users\goktu\workspace\datacops"

# Encoding notebook’tan gelen dosyalar
X_train_ready = load_npz(os.path.join(BASE_DIR, "X_train_ready.npz"))
X_test_ready  = load_npz(os.path.join(BASE_DIR, "X_test_ready.npz"))
feature_names = joblib.load(os.path.join(BASE_DIR, "feature_names.pkl"))

# Ölçeklenmiş train/test dataframe’leri
df_train = pd.read_csv(os.path.join(BASE_DIR, "train_scaled.csv"))
df_test  = pd.read_csv(os.path.join(BASE_DIR, "test_scaled.csv"))

print("X_train_ready:", X_train_ready.shape)
print("X_test_ready :", X_test_ready.shape)
print("Feature sayısı:", len(feature_names))


X_train_ready: (848024, 1786)
X_test_ready : (147207, 1786)
Feature sayısı: 1786


In [2]:
import numpy as np

# --- Hedef kolon --- 
# İstersen "AREA" yapabilirsin:
target_col = "Crm Cd"   # veya: target_col = "AREA"

is_classification = df_train[target_col].dtype == "object" or df_train[target_col].nunique() < 30

if is_classification:
    y_train = df_train[target_col].astype(str)
    y_test  = df_test[target_col].astype(str)
else:
    y_train = df_train[target_col].astype(float)
    y_test  = df_test[target_col].astype(float)

print("Classification mı?:", is_classification)
print("y_train shape:", y_train.shape)
print("y_test  shape:", y_test.shape)
print("Sınıf sayısı:", df_train[target_col].nunique())


Classification mı?: False
y_train shape: (848024,)
y_test  shape: (147207,)
Sınıf sayısı: 138


In [3]:
from scipy.sparse import issparse

def nan_to_zero_sparse(X):
    X = X.tocsr(copy=True)
    if X.data.size:
        mask = np.isnan(X.data)
        if mask.any():
            X.data[mask] = 0.0
    return X

X_train_ready = nan_to_zero_sparse(X_train_ready)
X_test_ready  = nan_to_zero_sparse(X_test_ready)
print("NaN temizliği tamam ✅")


NaN temizliği tamam ✅


In [4]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

# K1: ilk eleme sonrası tutulacak özellik sayısı
K1 = min(2000, X_train_ready.shape[1])
print(f"İlk aşamada {K1} özellik seçilecek...")

if is_classification:
    selector1 = SelectKBest(score_func=f_classif, k=K1)
else:
    selector1 = SelectKBest(score_func=f_regression, k=K1)

Xtr_k1 = selector1.fit_transform(X_train_ready, y_train)
Xte_k1 = selector1.transform(X_test_ready)

idx_k1 = np.where(selector1.get_support())[0]
feat_k1 = [feature_names[i] for i in idx_k1]

print("K1 sonrası X_train:", Xtr_k1.shape)
print("K1 sonrası X_test :", Xte_k1.shape)


İlk aşamada 1786 özellik seçilecek...
K1 sonrası X_train: (848024, 1786)
K1 sonrası X_test : (147207, 1786)


In [5]:
from sklearn.feature_selection import SelectKBest, f_classif

# K2: RFE sonunda kaç özellik kalsın?
K2 = min(200, Xtr_k1.shape[1])
print(f"RFE yerine SelectKBest ile {K2} özellik bırakılacak...")

# ---RFE YERİNE SelectKBest KULLANIYORUZ ---
#
# RFE, 4+ saat süren bir "Wrapper" yöntemdir.
# SelectKBest, 1-2 dakika süren hızlı bir "Filtre" yöntemidir.
# 'f_classif' (ANOVA F-testi), sınıflandırma için standart istatistiksel testtir.

print("SelectKBest (f_classif) istatistiksel testi çalıştırılıyor...")

# 'Xtr_k1' (1786 özellikli) ve 'y_train' verilerini kullanıyoruz
selector_k2 = SelectKBest(score_func=f_classif, k=K2)

# Modeli "eğit" (Aslında istatistiksel testi çalıştırır)
selector_k2.fit(Xtr_k1, y_train)

print("Test tamamlandı.")
# --- DEĞİŞİKLİK BİTTİ ---

# Kalan kodun RFE ile aynı çalışması için değişken isimlerini koru
mask_k2 = selector_k2.get_support()
idx_k2_within_k1 = np.where(mask_k2)[0]
idx_k2 = np.array(idx_k1)[idx_k2_within_k1]

feat_rfe = [feature_names[i] for i in idx_k2]

print(f"Seçilen özellik sayısı:", len(feat_rfe))
print("İlk 20 özellik:")
print(feat_rfe[:20])

RFE yerine SelectKBest ile 200 özellik bırakılacak...
SelectKBest (f_classif) istatistiksel testi çalıştırılıyor...
Test tamamlandı.
Seçilen özellik sayısı: 200
İlk 20 özellik:
['DR_NO', 'TIME OCC', 'Rpt Dist No', 'Vict Age', 'LAT', 'LON', 'Date Rptd_2020-01-02', 'Date Rptd_2020-01-29', 'Date Rptd_2020-02-28', 'Date Rptd_2020-03-29', 'Date Rptd_2020-03-30', 'Date Rptd_2020-04-19', 'Date Rptd_2020-04-24', 'Date Rptd_2020-05-04', 'Date Rptd_2020-05-11', 'Date Rptd_2020-05-24', 'Date Rptd_2020-05-27', 'Date Rptd_2020-05-30', 'Date Rptd_2020-05-31', 'Date Rptd_2020-06-01']


In [15]:
from scipy.sparse import save_npz

# 'idx_k2' (200 özellik) kullanarak matrisleri filtrele
X_train_rfe = X_train_ready[:, idx_k2]
X_test_rfe = X_test_ready[:, idx_k2]

# Yeni matrisleri kaydet
save_npz(os.path.join(BASE_DIR, "X_train_rfe.npz"), X_train_rfe)
save_npz(os.path.join(BASE_DIR, "X_test_rfe.npz"), X_test_rfe)

# İsimleri ve indeksleri kaydet
joblib.dump(idx_k2, os.path.join(BASE_DIR, "rfe_feature_indices.pkl"))
joblib.dump(feat_rfe, os.path.join(BASE_DIR, "rfe_feature_names.pkl"))

print("RFE (SelectKBest) matrisleri ve isimler kaydedildi ✅")
print(f"X_train_rfe: {X_train_rfe.shape}")
print(f"X_test_rfe: {X_test_rfe.shape}")

RFE (SelectKBest) matrisleri ve isimler kaydedildi ✅
X_train_rfe: (848024, 200)
X_test_rfe: (147207, 200)
